In [1]:
import pandas as pd

# Loading consolidated data

In [2]:
userprofile = pd.read_csv('WeightLoss/userprofile.csv')
valid_users = pd.read_csv("ProcessedData/valid_user_info_consolidated.csv", usecols = ['user_id'])

# Loading networks

In [3]:
friend_consolidated = pd.read_csv('WeightLoss/network/friend_consolidated.csv')

In [4]:
comment_rela = pd.read_csv('WeightLoss/network/comment_rela.csv')

In [5]:
mention_rela = pd.read_csv('WeightLoss/network/mention_rela.csv')
mention_rela.dropna(subset=['who-mention'], inplace=True) # remove rows with NaN in 'who-mention'
mention_rela['who-mention'] = mention_rela['who-mention'].astype('int64')

# Working with PyTorch

In [6]:
import torch
from torch_geometric.data import Data

In [7]:
# iteratively filters out low degree nodes from a graph
# returns list of nodes of degree at least d
# **note this runs out of memory if run directly on friend_net, would need an iterative solution maybe
def filter_low_degree_nodes(graph, d=5):
    # get the nodes and their degrees
    included_nodes, degrees = torch.unique(graph.edge_index[0], return_counts=True)
    print("Current Size:", included_nodes.shape)
    # generate a mask that is True for degrees >= d and False otherwise
    filter_mask = torch.ge(degrees,d)
    if(torch.unique(filter_mask).shape[0] != 1): # check that the mask still contains both True and False, otherwise done
        # filter the nodes based on the mask
        filtered_nodes = torch.masked_select(included_nodes,filter_mask)
        # generate the subgraph on the filtered nodes
        subgraph = graph.subgraph(filtered_nodes)
        # recursive call: now filter again based on the new subgraph
        # returns indices on what was passed in
        subgraph_nodes = filter_low_degree_nodes(subgraph)
        # convert back to original node names
        included_nodes = filtered_nodes[subgraph_nodes]
    # return the new included_nodes
    return included_nodes

In [8]:
num_nodes = userprofile["user_id"].max() # max id

# convert data to appropriate form and generate an undirected PyG graph
friend_edge_index = torch.from_numpy(friend_consolidated.iloc[:,[1,0]].to_numpy())
friend_net = Data(edge_index=friend_edge_index.t().contiguous(),num_nodes=num_nodes)
friend_net.coalesce()

comment_edge_index = torch.from_numpy(comment_rela.iloc[:,[0,1]].to_numpy())
comment_net = Data(edge_index=comment_edge_index.t().contiguous(),num_nodes=num_nodes)
comment_net.coalesce()


mention_edge_index = torch.from_numpy(mention_rela.iloc[:,[0,1]].to_numpy())
mention_net = Data(edge_index=mention_edge_index.t().contiguous(),num_nodes=num_nodes)
mention_net.coalesce()

Data(edge_index=[2, 2169263], num_nodes=12338632)

In [9]:
# filter out the low degree nodes
def checker(d=5):
    test_nets = [friend_net.subgraph(cur_nodes), comment_net.subgraph(cur_nodes), mention_net.subgraph(cur_nodes)]
    
    for test_net in test_nets:
        _, degs = torch.unique(test_net.edge_index[0], return_counts=True)
        unique_degs = torch.unique(degs)
        if unique_degs.min() < d: 
            print("Failed...")
            return False
    #
    print("Done!")
    return True
    
i = 0
hard_loop_cap = 10
cur_nodes = torch.from_numpy(valid_users.to_numpy().ravel())
min_deg = 5
while(i < hard_loop_cap):
    print("Filtering on comment_net:")
    comment_sub_net = comment_net.subgraph(cur_nodes)
    nodes_after_filter_on_comment_net = cur_nodes[filter_low_degree_nodes(comment_sub_net,min_deg)]
    print("Nodes after filter on comment net:", nodes_after_filter_on_comment_net)
    print("Filtering on mention_net:")
    mention_sub_net = mention_net.subgraph(nodes_after_filter_on_comment_net)
    nodes_after_filter_on_mention_net = nodes_after_filter_on_comment_net[filter_low_degree_nodes(mention_sub_net,min_deg)]
    print("Nodes after filter on mention net:", nodes_after_filter_on_mention_net)
    print("Filtering on friend_net:")
    friend_sub_net = friend_net.subgraph(nodes_after_filter_on_mention_net)
    cur_nodes = nodes_after_filter_on_mention_net[filter_low_degree_nodes(friend_sub_net,min_deg)]
    print("Current nodes:", cur_nodes)
    print("Checking...")
    if checker(min_deg): break
    i += 1
#
final_nodes = cur_nodes

Filtering on comment_net:
Current Size: torch.Size([422194])
Current Size: torch.Size([103543])
Current Size: torch.Size([92929])
Current Size: torch.Size([91733])
Current Size: torch.Size([91551])
Current Size: torch.Size([91517])
Current Size: torch.Size([91513])
Nodes after filter on comment net: tensor([       4,       10,       27,  ..., 12309780, 12318685, 12319222])
Filtering on mention_net:
Current Size: torch.Size([73393])
Current Size: torch.Size([41194])
Current Size: torch.Size([36450])
Current Size: torch.Size([35474])
Current Size: torch.Size([35247])
Current Size: torch.Size([35178])
Current Size: torch.Size([35159])
Current Size: torch.Size([35155])
Current Size: torch.Size([35154])
Current Size: torch.Size([35152])
Nodes after filter on mention net: tensor([      10,       31,       33,  ..., 12241455, 12253028, 12257012])
Filtering on friend_net:
Current Size: torch.Size([34882])
Current Size: torch.Size([34176])
Current Size: torch.Size([34174])
Current nodes: tensor

In [10]:
# get final networks
final_friend_net = friend_net.subgraph(final_nodes)
final_comment_net = comment_net.subgraph(final_nodes)
final_mention_net = mention_net.subgraph(final_nodes)

# Saving Data

In [11]:
directory = "ProcessedData"
# write final nodes to file
final_nodes_path = directory + "/included_users_(sorted).csv"
final_nodes_DF = pd.DataFrame(final_nodes.numpy(),columns=['user_id'])
final_nodes_DF.index.name = "Indices"
final_nodes_DF.to_csv(final_nodes_path)

In [12]:
# write filtered network edgelists to files
friend_edge_list_path = directory + "/friend_edges_directed.csv"
friend_edge_list_DF = pd.DataFrame(final_friend_net.edge_index.t().numpy(),columns=['follower_id','user_id'])
friend_edge_list_DF.to_csv(friend_edge_list_path, index = False)

In [13]:
comment_edge_list_path = directory + "/comment_edges_directed.csv"
comment_edge_list_DF = pd.DataFrame(final_comment_net.edge_index.t().numpy(),columns=['who-comment','comment-who'])
comment_edge_list_DF.to_csv(comment_edge_list_path, index = False)

In [14]:
mention_edge_list_path = directory + "/mention_edges_directed.csv"
mention_edge_list_DF = pd.DataFrame(final_mention_net.edge_index.t().numpy(),columns=['who-mention','mention-who'])
mention_edge_list_DF.to_csv(mention_edge_list_path, index = False)

# Getting Undirected Versions of Graphs

In [15]:
import numpy as np

# adds in reverse of all edges in edge_index
def to_undirected(edge_index):
    src, target = edge_index
    new_src= torch.cat([src, target])
    new_target = torch.cat([target, src])
    return torch.stack([new_src, new_target], dim=0)

# write undirected friend graph
friend_edge_list_path_undir = directory + "/friend_edges_undirected.csv"
friend_edge_list_np_undir = np.unique(to_undirected(final_friend_net.edge_index).t().numpy(),axis=0)
friend_edge_list_DF_undir = pd.DataFrame(friend_edge_list_np_undir,columns=['follower_id','user_id'])
friend_edge_list_DF_undir.to_csv(friend_edge_list_path_undir, index = False)

In [16]:
comment_edge_list_path_undir = directory + "/comment_edges_undirected.csv"
comment_edge_list_np_undir = np.unique(to_undirected(final_comment_net.edge_index).t().numpy(),axis=0)
comment_edge_list_DF_undir = pd.DataFrame(comment_edge_list_np_undir,columns=['who-comment','comment-who'])
comment_edge_list_DF_undir.to_csv(comment_edge_list_path_undir, index = False)

In [17]:
mention_edge_list_path_undir = directory + "/mention_edges_undirected.csv"
mention_edge_list_np_undir = np.unique(to_undirected(final_mention_net.edge_index).t().numpy(),axis=0)
mention_edge_list_DF_undir = pd.DataFrame(mention_edge_list_np_undir,columns=['who-mention','mention-who'])
mention_edge_list_DF_undir.to_csv(mention_edge_list_path_undir, index = False)

# Testing

In [18]:
# network degrees checker
#_, degs = torch.unique(final_mention_net.edge_index[0], return_counts=True)
#unique_degs,counts = torch.unique(degs, return_counts=True)
#print(unique_degs, counts)